In [1]:
import matplotlib.font_manager

In [2]:
fm = matplotlib.font_manager.json_load("C:\\Users\\Martin\\.matplotlib\\fontlist-v330.json")
print(fm.findfont("serif", rebuild_if_missing=False))
fm.findfont("serif", fontext="afm", rebuild_if_missing=False)

C:\Users\Martin\anaconda3\envs\bakalarka\Lib\site-packages\matplotlib\mpl-data\fonts\ttf\DejaVuSerif.ttf


'C:\\Users\\Martin\\anaconda3\\envs\\bakalarka\\Lib\\site-packages\\matplotlib\\mpl-data\\fonts\\afm\\pncr8a.afm'

In [3]:
from matplotlib import rc
rc('text', usetex=True)
rc('font', **{'family': 'serif', 'serif': ['Times-Roman']})

In [4]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import pickle
from tqdm import tqdm
import random
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
torch.set_default_dtype(torch.float64)
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as GraphLoader

import seaborn as sns
import matplotlib.pyplot as plt
import os

import pycuber as pc

In [5]:
from classes.cube_classes import Cube3State, Cube3
from generate.generate_states import ids_to_color
from utils.random_seed import seed_worker, seed_all, init_weights
from search.search import load_model, create_loader, get_train_test_set

In [6]:
import pytorch_classes.graph_dataset as gd
import pytorch_classes.color_dataset as cd

In [7]:
from explain.visualize_graph import visualize_graph

# Load Data

In [8]:
TEST_SIZE = 0.1
MAX_DISTANCE = 3
FILE = '5_moves_dataset_single.pkl'
pandas_reader = pd.read_pickle

In [9]:
df = pandas_reader(f'data/processed/{FILE}')
df['colors'] = df['colors'].map(np.array)
df

,state,colors,class_id,distance,generator
0,<classes.cube_classes.Cube3State object at 0x0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...",0,1,[[U']]
1,<classes.cube_classes.Cube3State object at 0x0...,"[0, 0, 0, 0, 0, 0, 5, 5, 5, 1, 1, 1, 1, 1, 1, ...",0,1,[[R']]
2,<classes.cube_classes.Cube3State object at 0x0...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, ...",0,1,[[D']]
3,<classes.cube_classes.Cube3State object at 0x0...,"[4, 4, 4, 0, 0, 0, 0, 0, 0, 5, 5, 5, 1, 1, 1, ...",0,1,[[L']]
4,<classes.cube_classes.Cube3State object at 0x0...,"[0, 0, 3, 0, 0, 3, 0, 0, 3, 2, 1, 1, 2, 1, 1, ...",0,1,[[F']]
...,...,...,...,...,...
105041,<classes.cube_classes.Cube3State object at 0x0...,"[5, 2, 2, 3, 0, 0, 3, 5, 5, 2, 4, 3, 2, 1, 3, ...",2228,5,"[[L, F', U', B', U']]"
105042,<classes.cube_classes.Cube3State object at 0x0...,"[0, 4, 4, 0, 0, 1, 3, 5, 1, 0, 5, 5, 5, 1, 1, ...",2228,5,"[[D, R', F', L', F']]"
105043,<classes.cube_classes.Cube3State object at 0x0...,"[0, 0, 4, 0, 0, 1, 5, 3, 1, 1, 1, 4, 0, 1, 4, ...",2228,5,"[[L, D', F', U', F']]"
105044,<classes.cube_classes.Cube3State object at 0x0...,"[3, 4, 5, 0, 0, 5, 5, 5, 1, 0, 5, 2, 0, 1, 1, ...",2228,5,"[[U, L', F', R', F']]"


In [10]:
df = df[df['distance'] <= MAX_DISTANCE]
print(len(df.index))

1194


In [11]:
def create_dataset(dataframe, solved_state_colors, test_size, random_seed, dataset_func, split_type):
    train, test = get_train_test_set(dataframe, test_size, random_seed, split_type)

    test_inputs = test['colors'].tolist()
    test_targets = test['distance'].tolist()
    test_dataset = dataset_func(test_inputs, test_targets)
    
    train_inputs = train['colors'].tolist()  # + [solved_state_colors]
    train_targets = train['distance'].tolist()  # + [0]
    train_dataset = dataset_func(train_inputs, train_targets)
    
    return train_dataset, test_dataset

In [12]:
solved_colors = (Cube3().generate_goal_states(1)[0].colors // 9).tolist()
train_loaders, test_loaders, train_dfs, test_dfs, train_datasets, test_datasets = [], [], [], [], [], []
for rnd_seed in range(10):
    tr_loader, tst_loader, train, test = create_loader(
        df, solved_colors, TEST_SIZE, rnd_seed, dataset_func=gd.create_data_list, dataloader_cls=GraphLoader, split_type='ratio'
    )
    tr_set, tst_set = create_dataset(
        df, solved_colors, TEST_SIZE, rnd_seed, dataset_func=gd.create_data_list, split_type='ratio'
    )
    train_loaders.append(tr_loader)
    test_loaders.append(tst_loader)
    train_dfs.append(train)
    test_dfs.append(test)
    train_datasets.append(tr_set)
    test_datasets.append(tst_set)

120it [00:00, 203.59it/s]
1075it [00:02, 372.40it/s]
120it [00:00, 390.48it/s]
1074it [00:02, 371.32it/s]
120it [00:00, 374.42it/s]
1075it [00:02, 369.83it/s]
120it [00:00, 375.59it/s]
1074it [00:02, 365.27it/s]
120it [00:00, 324.45it/s]
1075it [00:03, 350.67it/s]
120it [00:00, 369.25it/s]
1074it [00:03, 328.26it/s]
120it [00:00, 315.40it/s]
1075it [00:03, 316.85it/s]
120it [00:00, 299.85it/s]
1074it [00:03, 328.71it/s]
120it [00:00, 332.94it/s]
1075it [00:03, 345.29it/s]
120it [00:00, 346.63it/s]
1074it [00:03, 325.73it/s]
120it [00:00, 378.26it/s]
1075it [00:02, 369.25it/s]
120it [00:00, 363.50it/s]
1074it [00:02, 367.23it/s]
120it [00:00, 382.44it/s]
1075it [00:02, 369.98it/s]
120it [00:00, 363.97it/s]
1074it [00:03, 354.71it/s]
120it [00:00, 364.67it/s]
1075it [00:02, 379.02it/s]
120it [00:00, 347.96it/s]
1074it [00:02, 371.89it/s]
120it [00:00, 351.88it/s]
1075it [00:02, 368.48it/s]
120it [00:00, 364.20it/s]
1074it [00:02, 369.50it/s]
120it [00:00, 382.21it/s]
1075it [00:02, 374.3

# Load Models

In [13]:
class GraphModelWrapper(nn.Module):
    def __init__(self, model):
        super(GraphModelWrapper, self).__init__()
        self.model = model
        self.model.batch_norm = False
        self.double()
    
    def forward(self, x, edge_index, edge_attr, y):
        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
        return self.model.forward(data).squeeze(dim=0)

In [14]:
DATASET = '5moves'
MODEL = 'SymEqNet'
CHECKPOINT_DIR = f'checkpoints/{DATASET}/{MODEL}/'

In [15]:
models = []
for rnd_seed in range(10):
    model_path = os.path.join(CHECKPOINT_DIR, f'model_rs{rnd_seed}_ts{TEST_SIZE:0.1f}.pth')
    model = load_model(model_path)
    model = GraphModelWrapper(model)
    
    models.append(model)

In [16]:
color_mapping = ['blue', 'green', 'yellow', 'white', 'orange', 'red']

In [17]:
from torch_geometric.explain import Explainer
from torch_geometric.explain.metric import fidelity, unfaithfulness

In [52]:
def get_explainers(expl_algo_obj, explanation_type, train_epochs=None, explain_node_features=True, topk=10):
    explainers = []

    for rnd_seed in range(10):
        explainer = Explainer(
            model=models[rnd_seed],
            algorithm=expl_algo_obj,
            explanation_type=explanation_type,
            node_mask_type='attributes' if explain_node_features else None,
            edge_mask_type='object',
            model_config=dict(
                mode='regression',
                task_level='graph',
                return_type='raw',
            ),
            # Include only the top 10 most important edges:
            threshold_config=dict(threshold_type='topk', value=topk),
        )
    
        if train_epochs is not None:
            for epoch in tqdm(range(train_epochs)):
                for batch in train_loaders[rnd_seed]:
                    models[rnd_seed].eval()
                    loss = explainer.algorithm.train(
                        epoch, models[rnd_seed], batch.x, batch.edge_index, edge_attr=batch.edge_attr, y=batch.y.double(), target=batch.y.double()
                    )
        explainers.append(explainer)
    return explainers

In [19]:
def explain_and_visualize(explainers, explainer_folder, explanation_type, max_distance, explain_node_features=True):
    os.makedirs(f'imgs/explain/{MODEL}/{explainer_folder}/{explanation_type}', exist_ok=True)
    for rnd_seed in tqdm(range(10)):
        dataset = train_datasets[rnd_seed]
        curr_df = train_dfs[rnd_seed].copy()
        curr_df['processed'] = dataset

        distance_dfs = [
            curr_df[curr_df['distance'] == i] for i in range(1, max_distance + 1)
        ]

        for i, dist_df in enumerate(distance_dfs):
            datapoint = dist_df.iloc[0]
            colors = datapoint['colors']
            print(colors)
            generator = datapoint['generator'][0]
            data_input = datapoint['processed']
            node_labels = [f'{color_mapping[x]}-{y}' for x, y in zip(colors, datapoint.state.colors)]  # Nefunguje <{color_mapping[x]}<sub>{y}</sub>>
            explanation = explainers[rnd_seed](data_input.x, data_input.edge_index, edge_attr=data_input.edge_attr, y=data_input.y.double(), 
                                               target=data_input.y.double() if explanation_type == 'phenomenon' else None)
            # print(explanation.edge_mask)
            # explanation.visualize_feature_importance(top_k=10)
            colors = list(map(str, colors.tolist()))
            # explanation.visualize_graph(path=f'imgs/explain/{MODEL}/{explainer_folder}/dist{i+1}_rnd{rnd_seed}.pdf', backend="graphviz", node_labels=node_labels)
            visualize_graph(explanation, path=f'imgs/explain/{MODEL}/{explainer_folder}/{explanation_type}/dist{i+1}_rnd{rnd_seed}.pdf', backend="graphviz", node_labels=node_labels)
            visualize_graph(explanation, path=f'imgs/explain/{MODEL}/{explainer_folder}/{explanation_type}/dist{i+1}_rnd{rnd_seed}.png', backend="graphviz", node_labels=node_labels)
            
            if explain_node_features:
                explanation.visualize_feature_importance(top_k=5, path=f'imgs/explain/{MODEL}/{explainer_folder}/{explanation_type}/dist{i+1}_rnd{rnd_seed}_feat_imp.pdf')
                explanation.visualize_feature_importance(top_k=5, path=f'imgs/explain/{MODEL}/{explainer_folder}/{explanation_type}/dist{i+1}_rnd{rnd_seed}_feat_imp.png')
            
            rot_cube = pc.Cube()
            rot_cube(' '.join(generator))
            print(f'Random seed {rnd_seed}, Distance {i+1}')
            display(rot_cube)
            
#             unfaithfullness = unfaithfulness(explainers[rnd_seed], explanation)
#             print(f'Unfaithfullness: {unfaithfullness}')

# GNNExplainer

In [20]:
from torch_geometric.explain import GNNExplainer

## 1. Phenomenon

In [21]:
gnn_explainers = get_explainers(GNNExplainer(epochs=200), 'phenomenon')

In [22]:
explain_and_visualize(gnn_explainers, 'gnn_explainer', 'phenomenon', max_distance=MAX_DISTANCE)
pc_mapper = {''}

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[5 5 5 0 0 0 0 0 0 4 4 4 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 0 0 0 1 1 1 5 5 5 5 5 5]
Random seed 0, Distance 1


[3 0 0 3 0 0 4 0 0 1 1 2 1 1 2 1 1 5 5 2 2 5 2 2 0 0 0 1 1 1 4 3 3 4 3 3 2
 4 4 2 4 4 2 4 4 5 5 5 5 5 5 3 3 3]
Random seed 0, Distance 2


[3 4 2 3 0 0 4 0 0 3 5 2 1 1 2 1 1 2 5 5 5 2 2 2 4 0 0 5 1 1 4 3 3 4 3 3 2
 4 4 2 4 4 1 1 1 0 5 5 0 5 5 0 3 3]
Random seed 0, Distance 3


 10%|████████▎                                                                          | 1/10 [00:14<02:06, 14.05s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:27<00:09,  9.44s/it]

[4 4 4 0 0 0 0 0 0 5 5 5 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 1 1 1 0 0 0 5 5 5 5 5 5]
Random seed 9, Distance 1


[3 4 4 0 0 0 0 0 0 5 5 2 1 1 1 1 1 1 2 2 2 2 2 2 4 4 4 3 3 5 3 3 5 3 3 5 4
 4 3 4 4 3 1 1 1 0 0 0 5 5 2 5 5 2]
Random seed 9, Distance 2


[0 0 0 0 0 0 5 5 5 4 4 4 1 1 1 1 1 1 3 2 2 3 2 2 3 2 2 2 3 3 2 3 3 2 3 3 5
 0 0 5 4 4 1 4 4 0 5 5 4 5 5 4 1 1]
Random seed 9, Distance 3


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:37<00:00,  9.76s/it]


## 2. Model

In [23]:
gnn_explainers = get_explainers(GNNExplainer(epochs=200), 'model')

In [24]:
explain_and_visualize(gnn_explainers, 'gnn_explainer', 'model', max_distance=MAX_DISTANCE)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[5 5 5 0 0 0 0 0 0 4 4 4 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 0 0 0 1 1 1 5 5 5 5 5 5]
Random seed 0, Distance 1


[3 0 0 3 0 0 4 0 0 1 1 2 1 1 2 1 1 5 5 2 2 5 2 2 0 0 0 1 1 1 4 3 3 4 3 3 2
 4 4 2 4 4 2 4 4 5 5 5 5 5 5 3 3 3]
Random seed 0, Distance 2


[3 4 2 3 0 0 4 0 0 3 5 2 1 1 2 1 1 2 5 5 5 2 2 2 4 0 0 5 1 1 4 3 3 4 3 3 2
 4 4 2 4 4 1 1 1 0 5 5 0 5 5 0 3 3]
Random seed 0, Distance 3


 10%|████████▎                                                                          | 1/10 [00:07<01:11,  7.94s/it]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [01:22<00:09,  9.17s/it]

[4 4 4 0 0 0 0 0 0 5 5 5 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 1 1 1 0 0 0 5 5 5 5 5 5]
Random seed 9, Distance 1


[3 4 4 0 0 0 0 0 0 5 5 2 1 1 1 1 1 1 2 2 2 2 2 2 4 4 4 3 3 5 3 3 5 3 3 5 4
 4 3 4 4 3 1 1 1 0 0 0 5 5 2 5 5 2]
Random seed 9, Distance 2


[0 0 0 0 0 0 5 5 5 4 4 4 1 1 1 1 1 1 3 2 2 3 2 2 3 2 2 2 3 3 2 3 3 2 3 3 5
 0 0 5 4 4 1 4 4 0 5 5 4 5 5 4 1 1]
Random seed 9, Distance 3


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [01:32<00:00,  9.26s/it]


# PGExplainer

## 1. Phenomenon

In [232]:
from torch_geometric.explain import PGExplainer

In [164]:
EPOCHS = 10
pg_explainers_ph = get_explainers(PGExplainer(epochs=EPOCHS, lr=0.003), 'phenomenon', train_epochs=EPOCHS, explain_node_features=False)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [02:22<00:00, 14.26s/it]


In [165]:
explain_and_visualize(pg_explainers_ph, 'pg_explainer', 'phenomenon', max_distance=MAX_DISTANCE, explain_node_features=False)

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

[5 5 5 0 0 0 0 0 0 4 4 4 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 0 0 0 1 1 1 5 5 5 5 5 5]
Random seed 0, Distance 1


[3 0 0 3 0 0 4 0 0 1 1 2 1 1 2 1 1 5 5 2 2 5 2 2 0 0 0 1 1 1 4 3 3 4 3 3 2
 4 4 2 4 4 2 4 4 5 5 5 5 5 5 3 3 3]
Random seed 0, Distance 2


[3 4 2 3 0 0 4 0 0 3 5 2 1 1 2 1 1 2 5 5 5 2 2 2 4 0 0 5 1 1 4 3 3 4 3 3 2
 4 4 2 4 4 1 1 1 0 5 5 0 5 5 0 3 3]
Random seed 0, Distance 3


 10%|████████▎                                                                          | 1/10 [00:00<00:04,  1.93it/s]

...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:04<00:00,  1.90it/s]

[4 4 4 0 0 0 0 0 0 5 5 5 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 3 3 3 3 3 3 3 3 3 4
 4 4 4 4 4 1 1 1 0 0 0 5 5 5 5 5 5]
Random seed 9, Distance 1


[3 4 4 0 0 0 0 0 0 5 5 2 1 1 1 1 1 1 2 2 2 2 2 2 4 4 4 3 3 5 3 3 5 3 3 5 4
 4 3 4 4 3 1 1 1 0 0 0 5 5 2 5 5 2]
Random seed 9, Distance 2


[0 0 0 0 0 0 5 5 5 4 4 4 1 1 1 1 1 1 3 2 2 3 2 2 3 2 2 2 3 3 2 3 3 2 3 3 5
 0 0 5 4 4 1 4 4 0 5 5 4 5 5 4 1 1]
Random seed 9, Distance 3


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.87it/s]


## 2. Model

In [168]:
# EPOCHS = 10
# pg_explainers_m = get_explainers(PGExplainer(epochs=EPOCHS, lr=0.003), 'model', train_epochs=EPOCHS, explain_node_features=False)

In [169]:
# explain_and_visualize(pg_explainers_m, 'pg_explainer', 'model', max_distance=MAX_DISTANCE, explain_node_features=False)

# Explaining Single Move

In [25]:
from generate.generate_states import char_to_move_index

In [55]:
def explain_single_move(explainers, explainer_folder, explanation_type, move):
    cube = Cube3()
    solved_state = cube.generate_goal_states(1)[0]
    print(char_to_move_index(move))
    dist1_state = cube.next_state([solved_state], char_to_move_index(move))[0][0]
    solved_colors = (solved_state.colors // 9).tolist()
    dist1_colors = (dist1_state.colors // 9).tolist()
    print(dist1_colors)
    
    rot_cube = pc.Cube()
    display(rot_cube)
    rot_cube(move)
    display(rot_cube)
    
    state_list = [solved_state, dist1_state]
    single_move_dataset = gd.create_data_list([solved_colors, dist1_colors], [0, 1])
    
    os.makedirs(f'imgs/explain_single/{MODEL}/{explainer_folder}/{explanation_type}', exist_ok=True)
    for rnd_seed in tqdm(range(10)):
        for i, (data_input, state) in enumerate(zip(single_move_dataset, state_list)):
            explanation = explainers[rnd_seed](data_input.x, data_input.edge_index, edge_attr=data_input.edge_attr, y=data_input.y.double(), 
                                               target=data_input.y.double() if explanation_type == 'phenomenon' else None)
            node_labels = [f'{color_mapping[x]}-{y}' for x, y in zip(state.colors // 9, state.colors)]
            # print(explanation.edge_mask)
            visualize_graph(explanation, path=f'imgs/explain_single/{MODEL}/{explainer_folder}/{explanation_type}/pdf/dist{i}_rnd{rnd_seed}.pdf', backend="graphviz", node_labels=node_labels)
            visualize_graph(explanation, path=f'imgs/explain_single/{MODEL}/{explainer_folder}/{explanation_type}/png/dist{i}_rnd{rnd_seed}.png', backend="graphviz", node_labels=node_labels)

In [64]:
gnn_explainers = get_explainers(GNNExplainer(epochs=200), 'phenomenon', topk=10)
# ['blue', 'green', 'yellow', 'white', 'orange', 'red']

In [65]:
explain_single_move(gnn_explainers, 'gnn_explainer', 'phenomenon', 'R')

9
[0, 0, 3, 0, 0, 3, 0, 0, 3, 2, 1, 1, 2, 1, 1, 2, 1, 1, 0, 0, 0, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5]


2it [00:00, 611.73it/s]
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:30<00:00,  3.09s/it]


# Architecture-Based Interpretations

In [72]:
MOVE = 'R'
TOPK = 10
cube = Cube3()
solved_state = cube.generate_goal_states(1)[0]
print(char_to_move_index(MOVE))
dist1_state = cube.next_state([solved_state], char_to_move_index(MOVE))[0][0]
solved_colors = (solved_state.colors // 9).tolist()
dist1_colors = (dist1_state.colors // 9).tolist()
print(dist1_colors)

rot_cube = pc.Cube()
display(rot_cube)
rot_cube(MOVE)
display(rot_cube)

state_list = [solved_state, dist1_state]
single_move_dataset = gd.create_data_list([solved_colors, dist1_colors], [0, 1])

7
[0, 0, 0, 0, 0, 0, 5, 5, 5, 1, 1, 1, 1, 1, 1, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 0, 0, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 1, 1, 1]


2it [00:00, 277.84it/s]


In [114]:
def my_lexsort(x):
    return np.flip(x[np.lexsort(np.rot90(x))], axis=0)

def bmatrix(a):
    """Returns a LaTeX bmatrix

    :a: numpy array
    :returns: LaTeX bmatrix as a string
    """
    if len(a.shape) > 2:
        raise ValueError('bmatrix can at most display two dimensions')
    lines = str(a).replace('[', '').replace(']', '').splitlines()
    rv = [r'\begin{bmatrix}']
    rv += ['  ' + ' & '.join(l.split()) + r'\\' for l in lines]
    rv +=  [r'\end{bmatrix}']
    return '\n'.join(rv)

np.set_printoptions(precision=2)

In [115]:
for rnd_seed in tqdm(range(10)):
    for i, (data_input, state) in enumerate(zip(single_move_dataset, state_list)):
        input_repre = data_input.x.numpy()
        print('TOPK:')
#         print(my_lexsort(input_repre)[:TOPK])
        print(bmatrix(my_lexsort(input_repre)[:TOPK]))
        with torch.no_grad():
            output_repre = models[rnd_seed].model.conv1(data_input.x, data_input.edge_index, data_input.edge_attr)
            output_repre = output_repre.numpy()
#         print(my_lexsort(output_repre)[:TOPK])
        print(bmatrix(my_lexsort(output_repre)[:TOPK]))
        print('\n')
        print('Mean:')
        print(np.mean(input_repre))
        print(np.mean(output_repre))
        print('\n\n')
    print(100*'-')
    print('\n\n')

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 58.10it/s]

TOPK:
\begin{bmatrix}
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
  0. & 1. & 1. & 1. & 2. & 2. & 2. & 2. & 2.\\
\end{bmatrix}
\begin{bmatrix}
  0. & 14.71 & 14.71 & 14.71 & 21.54 & 21.54 & 28.02 & 28.02 & 28.02\\
  0. & 14.71 & 14.71 & 14.71 & 21.54 & 21.54 & 28.02 & 28.02 & 28.02\\
  0. & 14.71 & 14.71 & 14.71 & 21.54 & 21.54 & 28.02 & 28.02 & 28.02\\
  0. & 14.71 & 14.71 & 14.71 & 21.54 & 21.54 & 28.02 & 28.02 & 28.02\\
  0. & 14.71 & 14.71 & 14.71 & 21.54 & 21.54 & 28.02 & 28.02 & 28.02\\
  0. & 14.71 & 14.71 & 14.71 & 21.54 & 21.54 & 28.02 & 28.02 & 28.02\\
  0. & 14.71 & 14.71 & 14.71 & 21.54 & 21.54 & 28.02

## Distance 1 & 3

In [118]:
df1 = df[df['distance'] == 1]
dist1_dataset = gd.create_data_list(df1.colors.tolist(), df1.distance.tolist())

12it [00:00, 378.89it/s]


In [122]:
for rnd_seed in tqdm(range(10)):
    for i, data_input in enumerate(dist1_dataset):
        input_repre = data_input.x.numpy()
        print('TOPK:')
        print(my_lexsort(input_repre)[:TOPK])
#         print(bmatrix(my_lexsort(input_repre)[:TOPK]))
        with torch.no_grad():
            output_repre = models[rnd_seed].model.conv1(data_input.x, data_input.edge_index, data_input.edge_attr)
            output_repre = output_repre.numpy()
        print(my_lexsort(output_repre)[:TOPK])
#         print(bmatrix(my_lexsort(output_repre)[:TOPK]))
#         print('\n')
        print('Mean:')
        print(np.mean(input_repre))
        print(np.mean(output_repre))
        print('\n\n')
    print(100*'-')
    print('\n\n')

 40%|█████████████████████████████████▏                                                 | 4/10 [00:00<00:00, 39.38it/s]

TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



----------------------------------------------------------------------------------------------------



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.28595516708701

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 38.88it/s]

Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



----------------------------------------------------------------------------------------------------



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087025



TOPK:
Mean:
1.4979423868312758
24.285955167087018



TOPK:
Mean:
1.4979423868312758
24.285955167087018



------------------------------------------------------

In [124]:
df3 = df[df['distance'] == 3]
dist3_dataset = gd.create_data_list(df3.colors.tolist(), df3.distance.tolist())

1068it [00:03, 343.08it/s]


In [128]:
means_in = []
means_out = []
for rnd_seed in tqdm(range(10)):
    for i, data_input in enumerate(dist3_dataset):
        input_repre = data_input.x.numpy()
        print('TOPK:')
        print(my_lexsort(input_repre)[:TOPK])
#         print(bmatrix(my_lexsort(input_repre)[:TOPK]))
        with torch.no_grad():
            output_repre = models[rnd_seed].model.conv1(data_input.x, data_input.edge_index, data_input.edge_attr)
            output_repre = output_repre.numpy()
#         print(my_lexsort(output_repre)[:TOPK])
#         print(bmatrix(my_lexsort(output_repre)[:TOPK]))
#         print('\n')
        print('Mean:')
        print(np.mean(input_repre))
        means_in.append(np.mean(input_repre))
        print(np.mean(output_repre))
        means_out.append(np.mean(output_repre))
        print('\n\n')
    print(100*'-')
    print('\n\n')

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.8353909465020577
36.02394636121728



TOPK:
Mean:
1.8353909465020577
36.02394636121728



TOPK:
Mean:
1.8353909465020577
36.02394636121727



TOPK:
Mean:
1.8353909465020577
36.02394636121727



TOPK:
Mean:
1.8353909465020577
36.02394636121728



TOPK:
Mean:
1.8353909465020577
36.02394636121728



TOPK:
Mean:
1.8353909465020577
36.02394636121728



TOPK:
Mean:


 10%|████████▎                                                                          | 1/10 [00:02<00:20,  2.24s/it]

Mean:
1.7818930041152263
35.450636069698774



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.781

 20%|████████████████▌                                                                  | 2/10 [00:04<00:17,  2.18s/it]

...

Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.59902665749

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:20<00:02,  2.32s/it]

Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



----------------------------------------------------------------------------------------------------



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
33.39049504773466



TOPK:
Mean:
1.7613168724279835
3

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:23<00:00,  2.31s/it]

Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.7818930041152263
35.45063606969877



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.599026657499905



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.728395061728395
33.5990266574999



TOPK:
Mean:
1.7283950617

In [129]:
print(f'Mean input: {np.mean(means_in)} +- {np.std(means_in)}')
print(f'Mean hidden: {np.mean(means_out)} +- {np.std(means_out)}')

Mean input: 1.7161880982105702 +- 0.07759310469839946
Mean hidden: 32.770470603059955 +- 2.599685860929955
